In [9]:
import simpy
import random
import statistics

# ---------------------------
# PARAMETERS
# ---------------------------
LAMBDA_ARRIVAL = 5        # 평균 5명/분 도착
MEAN_ID_TIME = 0.75       # ID체크 평균시간 (지수분포)
PERSONAL_MIN = 0.5        # 개인검색 최소
PERSONAL_MAX = 1.0        # 개인검색 최대
SIM_TIME = 5000           # 시뮬레이션 시간 (분)

# ---------------------------
# DATA COLLECTION
# ---------------------------
wait_times = []  # 전체 시스템 체류시간 저장

# ---------------------------
# FUNCTIONS
# ---------------------------
def passenger(env, name, id_checker, personal_queues):
    """각 승객의 흐름"""
    arrival_time = env.now

    # 1️⃣ ID 체크 단계
    with id_checker.request() as req:
        yield req
        service_time = random.expovariate(1.0 / MEAN_ID_TIME)
        yield env.timeout(service_time)

    # 2️⃣ 개인 검색 단계 (가장 짧은 줄 선택)
    shortest_queue = min(personal_queues, key=lambda q: len(q.queue))
    with shortest_queue.request() as req2:
        yield req2
        service_time2 = random.uniform(PERSONAL_MIN, PERSONAL_MAX)
        yield env.timeout(service_time2)

    # 종료 시점 기록
    total_time = env.now - arrival_time
    wait_times.append(total_time)

def passenger_generator(env, id_checker, personal_queues):
    """포아송 도착"""
    i = 0
    while True:
        i += 1
        yield env.timeout(random.expovariate(LAMBDA_ARRIVAL))
        env.process(passenger(env, f"Passenger {i}", id_checker, personal_queues))

In [10]:
# ---------------------------
# SIMULATION RUNNER
# ---------------------------
def run_simulation(n_id, n_personal, sim_time=SIM_TIME):
    random.seed(20251022)
    env = simpy.Environment()
    id_checker = simpy.Resource(env, capacity=n_id)
    personal_queues = [simpy.Resource(env, capacity=1) for _ in range(n_personal)]

    env.process(passenger_generator(env, id_checker, personal_queues))
    env.run(until=sim_time)

    avg_time = statistics.mean(wait_times)
    return avg_time

# ---------------------------
# EXPERIMENT
# ---------------------------
for id_checkers in [2, 3, 4, 5]:
    for personals in [3, 4, 5, 6]:
        wait_times.clear()
        avg = run_simulation(id_checkers, personals)
        print(f"ID={id_checkers}, Personal={personals} → Avg total time: {avg:.2f} min")

ID=2, Personal=3 → Avg total time: 1167.99 min
ID=2, Personal=4 → Avg total time: 1142.93 min
ID=2, Personal=5 → Avg total time: 1143.00 min
ID=2, Personal=6 → Avg total time: 1155.39 min
ID=3, Personal=3 → Avg total time: 518.32 min
ID=3, Personal=4 → Avg total time: 498.19 min
ID=3, Personal=5 → Avg total time: 462.25 min
ID=3, Personal=6 → Avg total time: 472.05 min
ID=4, Personal=3 → Avg total time: 478.22 min
ID=4, Personal=4 → Avg total time: 5.91 min
ID=4, Personal=5 → Avg total time: 4.32 min
ID=4, Personal=6 → Avg total time: 4.58 min
ID=5, Personal=3 → Avg total time: 498.31 min
ID=5, Personal=4 → Avg total time: 3.48 min
ID=5, Personal=5 → Avg total time: 2.24 min
ID=5, Personal=6 → Avg total time: 2.17 min
